# Chapter 5: Shortest path algorithms


*Selected Topics in Mathematical Optimization*

*2016-2017*

**Bernard De Baets**

**Michiel Stock**

**Raúl Pérez-Fernández**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## Representing graphs in Python

Graphs are discrete mathematical objects, represented as a tuple $G=(V, E)$ with $V$ the set of vertices (nodes, points) and $E$ the set of edges (links, arcs, lines) that connect the vertices.

Consider for example the graph below.

![A directed unweighted graph](Figures/unweighted_graph.png)

This graph contains five edges: A, B, C, D and E and nine vertices. Since each of the vertices has a direction from one node to another so we call this a **directed graph**.

Like most programming languages, Python does not provide direct support for graphs a data types. Graphs can however be represented by [**dictionaries**](https://docs.python.org/2/tutorial/datastructures.html#dictionaries). Dictionaries in Python store key-value pairs in a way such that accessing, insertion and deletion of new elements can be done very efficiently. The keys will represent the edges, while its associated values is the set of vertices to which depart from this node.

For example, the unweighted graph above is represented bij the following code.

In [ ]:
unweighted_graph = {'A' : set(['B']),
                   'B' : set(['C']),
                   'C' : set(['A', 'B', 'E']),
                   'D' : set(['A']),
                   'E' : set(['D', 'D'])  # set data structure only contains unique elements
                   }                     # only one node will be retained

Note that we store the outgoing vertices in a [**set**](https://docs.python.org/2/tutorial/datastructures.html#sets) data structure. This will allow us to perform certain operations faster. Note that of the two vertices starting for E only one is retained. 

In [ ]:
unweighted_graph['E']

In this chapter, we will focus on algorithms that process **weighted graphs**, i.e. graphs on which the vertices have a weight. For example:
![A weighted directed graph](Figures/weighted_graph.png)
We will store each vertex now as a tuple containing the weight and the edge it is connected to. For this example:

In [ ]:
weighted_graph = {'A' : set([(3, 'B')]),
                   'B' : set([(2, 'C')]),
                   'C' : set([(6, 'A'), (1, 'B'), (0.3, 'E')]),
                   'D' : set([(9, 'A')]),
                   'E' : set([(2.8, 'D'), (4.1, 'D'), (4.2, 'E')])  # two paths from E -> D
                    }

Storing a weighted vertex as `(weight, outgoing node)` has the advantage that the native `sort()` function in Python can be used to order vertices by weight, something which will be useful in the algorithms we will implement.

## Priority queues using heapq

In order to efficiently implement the algorithms from this chapter, we will need to make use of a **priority queue**. A priority queue is a data structure which is perhaps most easily explained using a small example.

A professor receives an enormous number of e-mails during the day. Because he cannot respond to them all into detail, the professor witholds only the 50 most important mails and processes them in the evening. Consider the following algorithm the prof might use to mananage his set of important mails during the day.

In [ ]:
from random import random

# simulate 10000 e-mails
# each e-mail is represented by an 'importance' and 
# an indice (the number of the mail)
emails = [(random(), i) for i in range(10000)]

# first five mails
emails[:5]

In [ ]:
# retain the 50 most important mails using sort

def top_k_sorted(k=50):
    top_set = [(-1e10, -1) for i in range(k)]  # initialize the to respond list
    for importance, id in emails:
        # assume that the least important mail is on position 0
        if top_set[0][0] < importance:
            top_set[0] = (importance, id)  # replace least important mail
            top_set.sort()  # sort
    return top_set

top_k_sorted()

Taking a closer look, this routine is very wasteful of the time of the poor professor! Each time he updates his list of to-respond mails, he has to completely sort it so that he can easily access his least important mail. Sorting a list has a time complexity of $\mathcal{O}(n\log(n))$ for a list of $n$ elements. Sorting the complete list is overkill, as the professor wants the most important mails of the list.

A more clever data structure for these types of tasks is a [**heap**](https://docs.python.org/2/library/heapq.html). A heap is a partially sorted list such that `heap[k] <= heap[2*k+1]` and `heap[k] <= heap[2*k+2]` for all `k`.

A heap is based on the idea of a tournament, as represented below (number represent indices, not values). 

```
                               0

              1                                 2

      3               4                5               6

  7       8       9       10      11      12      13      14

15 16   17 18   19 20   21 22   23 24   25 26   27 28   29 30
```
A list `a` that satisfies the heap property always has that `a[0] < a[1]` and `a[0] < a[2]`, while `a[1] < a[3]` and `a[1] < a[4]` etc.

The following operations can be done using a heap:
- `heappush`: adding an item to the heap while maintaining the heap property ($\mathcal{O}(\log(n))$)
- `heapop`: removing the item with the lowest value while maintaining the heap property ($\mathcal{O}(\log(n))$)
- `heappushpop`: replacing the item with the lowest value by another item while maintaining the heap property ($\mathcal{O}(\log(n))$)
- peeking: looking the item at position 0, which has the lowest value ($\mathcal{O}(1)$)

Heaps are implemented in `heapq` in Python. Consider our example.

In [ ]:
from heapq import heappop, heappush, heappushpop

def top_k_heaped(k=50):
    top_set = [(-1e10, -1) for i in range(k)]  # initialize the to respond list
    for importance, id in emails:
        # assume that the least important mail is on position 0
        if top_set[0][0] < importance:
            # replace least important mail while maintaining the heap property
            heappushpop(top_set, (importance, id))  
    return top_set

top_k_heaped()

Both algorithms return the same list of e-mails, though using a heap to implement a priority queue is more efficient in resources.

In [ ]:
%timeit top_k_sorted()

In [ ]:
%timeit top_k_heaped()

## Dijkstra's shortest path algorithm

Dijkstra's algorithm is a popular algorithm to find the shortest path between the nodes of a graph. The algorithm can be used in two ways:

- when both a source and a sink node are provided, the algorithm gives the list of nodes of the shortest path, together with the length (distance of the path).
- when only a source is given, the shortest distance between the source and all (accessable) nodes is returned together with a dictionary representing the tree of the shortest paths between the source and other nodes.

The pseudocode can be found below.

### Pseudocode of Dijkstra's shortest path algorithm
Source: Wikipedia.org
```
1  function Dijkstra(Graph, source, sink (optional)):
2      distance[source] := 0  // initialize a dictionary with distance
3                             // to source
4      for each vertex V in Graph: 
5          if V ≠ source
6              distance[V] := infinity
7      previous := empty dict   // stores the previous node in path
8
10     make empty priority queue Q for vertices to check
11     Q.add_with_priority(source, 0)  # first check the source
12
13
14     while Q is not empty:                     // The main loop
15         get node U with smallest distance from Q
16         if U is the sink (if provided) 
17              break the loop
17         for every neighbor V of U:
18             alternative := distance[U] + dist(U, V)
18             if alternative < distance[V]
19                 distance[V] := alternative
20                 previous[V] := U
21                 Q.add_with_priority(V, alternative)
22
23     if sink is given
24         reconstruct shortest path
25         return shortest path and length of path
26     else
27         return distance, previous
```

### Pseudocode of path reconstruction  algorithm
```
1 function reconstruct_path(previous, source, sink)
2    if the source is not in previous
3          return empty path
4    V := sink   \\ path is reconstructed backwards
5    path := [V]  \\ path is a list of nodes
6    while V is not source
7         V := previous node in path
8         add V in beginning of the path
9    return the path
```

In [ ]:
from numpy import inf

def dijkstra(graph, source, sink=None):
    """
    Implementation of Dijkstra's shortest path algorithm
    Inputs:
        - graph : dict representing the weighted graph
        - source : the source node
        - sink : the sink node (optional)
    Ouput:
        - distance : dict with the distances of the nodes to the source
        - came_from : dict with for each node the came_from node in the shortest
                    path from the source
    """
    # COMPLETE THIS
    
    if sink is None:
        return distance, came_from
    else:
        return reconstruct_path(came_from, source, sink), distance[sink]

In [ ]:
def reconstruct_path(came_from, source, sink):
    """
    Reconstruct the path from the output of the Dijkstra algorithm
    Inputs:
            - came_from : a dict with the came_from node in the path
            - source : the source node
            - sink : the sink node
    Ouput:
            - the shortest path from source to sink (list)
    """
    if sink not in came_from:
        return []
    # COMPLETE THIS
    return path

**Assignment 1: Dijkstra**

1. Complete the implementation for Dijstra's algorithm and the path reconstruction function
2. Find the shortest path from node A to I in the example graph below. Calculate the shortest distance from A to any other node. Compare both running times with `%timeit`.

![Example network](Figures/example_graph.png)

In [ ]:
# the above graph as an adjacency list
graph = {'A' : set([(2, 'B'), (3, 'D')]),
        'B' : set([(1, 'C'), (5, 'E')]),
        'C' : set([(2, 'D'), (1, 'E')]),
        'D' : set([(3, 'E')]),
        'E' : set([(2, 'B'), (3, 'F')]),
        'F' : set([(5, 'G'), (8, 'I')]),
        'G' : set([(2, 'H'), (5, 'I')]),
        'H' : set([(3, 'I')]),
        'I' : []}

In [ ]:
dijkstra(graph, 'A', 'I')

In [ ]:
dijkstra(graph, 'A')

In [ ]:
%timeit dijkstra(graph, 'A', 'I')

In [ ]:
%timeit dijkstra(graph, 'A')

## A* shortest path algorithm

It is hard to explain the algorithm into more depth than [here](http://theory.stanford.edu/~amitp/GameProgramming/) and [here](http://www.redblobgames.com/pathfinding/a-star/introduction.html).

Dijkstra's algorithm is guaranteed to find a shortest path on any (positively weighted) graph. But because no assumptions are made of the nature of the graph, Dijkstra's algorithm cannot exploit prior knowledge to speed up its search. Consider the problem of finding a shortest path on a grid using Dijkstra.

![Dijkstra's algorithm for grid with no obstacles.](Figures/Dijkstra.png)

In the figure above, it is clear that Dijkstra searches very inefficient: it tries to expand its seach horizon around the starting point in order to find the shortest path. Consider a 'greedy' algorithm which always tries to move as close to the goal as possible.

![Greedy search for grid with no obstacles.](Figures/best-first-search.png)

Much better! The algorithm always moves in the right direction. What happens when a direct path between the starting position and the goal is not possible?

![Dijkstra's algorithm for grid with an obstacle.](Figures/dijkstra-trap.png)

Dijkstra is guaranteed to find the optimal path, but again only after much exploring! Let us see of the greedy algorithm again fares better.

![Greedy search for grid with an obstacle.](Figures/best-first-search-trap.png)

We see again that the greedy algorithm does much less searching, but this time the path that is returned is not the shortest!

The A\* algorithm lets us have the best of both worlds: it is exact (like Dijkstra's algorithm), but it can also use a **heuristic** to speed up the search. In each iteration, the next neighbor $n$ of the current node that the algorithm considers is chosen based on the heuristic

$$
f(n) = g(n) + h(n)
$$

with $g(n)$ the cost of the path so far and $h(n)$ a heuristic that estimates the cost of the cost of the shortest path from $n$ to the goal. The heuristic $h(n)$ should satify two properties:
1. To guaranty that a shortest path is found, the heuristic should be **admissible**, i.e. it should never overestimate the true distance to goal.
2. Evaluating the heuristic should be cheap (it is either pre-computed or is a function).

So the A\* algorithm is basically the same as Dijkstra's algorithm, but with the main difference that the the latter chooses nodes to explore based on the distance from the starting node, while the latter chooses nodes **based on an estimate of distance to the goal**.

When using A\* to find the shortest path between two physical locations, each node corresponds to a point in space. A good heuristic is the (Euclidian) distance between the nodes, as this will always be a lower bound for the actual distance to travel to the goal.

Consider the following example of finding the path from node $a$ to node $h$.

![Example how A\* uses the heuristics](Figures/astarexample.png)

The shortest path so far is $[a, e]$ and the algorithm needs to choose to explore node $d$ or node $g$. Since the path of $ed$ is shorter than the path $eg$, Dijkstra will choose this node, even though it is farther removed from the goal $h$ (Dijkstra chooses a neighbor $n$ only based on the current path length form the starting node $g(n)$. The A\* algorithm will chose node $g$ to explore, because the estimated path length $f(e) = g(e) + h(e) > f(d)$ and hence will approach the goal. 

### Pseudocode of A\* shortest path algorithm
Source from Wikipedia (modified)
```
1  function Astar(Graph, source, sink, heuristic):
2      make empty priority queue Q for vertices to check
3      
4      add source to Q with priority f(source)
5          // use the estimated path length for priority
6              
7      distance[V] := 0   // initialize a dictionary with distance
8                           // to source
9      previous := empty dict   // stores the previous node in path
10
11     while Q is not empty:                     // The main loop
12         pop node U with lowest estimated path length to sink
13         if U is the sink
14              break the loop
15         for every neighbor V of U:
16             distance_source_U := distance[U] + dist(U, V)
17             if V not in distance or distance_source_U < distance[V]
18                 distance[V] := distance_source_U
19                 previous[V] := U
20                 heuristic_V_sink := distance[V] + g(V)
21                 Q.add_with_priority(V, heuristic_V_sink)
22
23     reconstruct shortest path
24     return shortest path and length of path
```

#### Representation of the graphs for physical locations

For the assignments above, we will work with graphs representing paths on a two-dimensional plane. 

Each node or location is a point on this plane, e.g. $(x_i, y_i)$. An edge or link between two locations is a tuple of two points: $((x_i, y_i),(x_j, y_j))$. In the link representation, the network is represented as just a list of these links. For example:

In [ ]:
from parcour_tools import add_path, load_links, plot_parcour, links_to_graph

links_example = [((1, 1), (1, 3)), ((1, 1), (2, 2)), ((1,3), (2, 2)),
                ((2, 2), (3, 1)), ((2, 2), (3, 3)), ((3, 1), (3, 3))]

Using the function ```links_to_graph``` we can change these links in the more familiar dictionary-based representation.

In [ ]:
graph_example = links_to_graph(links_example)
print(graph_example)

Which can be visualized using ```plot_parcour``` and we can use the function ```add_path``` to draw a path.

In [ ]:
fig, ax = plt.subplots()
plot_parcour(links_example, ax, line_width=20)  # uses the links representation as an input!
ax.set_ylim([0.5, 3.5])
ax.set_xlim([0.5, 3.5])

path_example = [(1, 1), (1, 3), (2, 2), (3, 1), (3, 3)]
add_path(path_example, ax, color='green')

**Assignment 2: A\***

1. Complete the implementation for the A\* algorithm.
2. Some parcours and mazes are given. Use A\* and Dijkstra to find the shortest paths between the lower left corner to the upper right corner. Use the function ```add_path``` to add the path to the figure.
3. Compare running time of the two shortest path algorithms using ```%timeit```.

In [ ]:
# functions for computing distances
manhattan_distance = lambda x, y : sum([abs(xi - yi) for xi, yi in zip(x, y)])
euclidian_distance = lambda x, y : sum([(xi - yi)**2 for xi, yi in zip(x, y)])**0.5

In [ ]:
def a_star(graph, source, sink, heuristic):
    """
    Implementation of the A* shortest path algorithm
    Inputs:
        - graph : dict representing the graph
        - source : the source node
        - sink : the sink node (optional)
        - heuristic : a function with the heuristic for the shortest path between two nodes
    Ouput:
        - distance : dict with the distances of the nodes to the source
        - came_from : dict with for each node the previous node in the shortest
                    path from the source
    """
    # COMPLETE THIS
    return

In [ ]:
# a parcour, white lines are the routes that can be followed
# find the shortest path from (0, 0) to (99, 99)
parcour = load_links('Data/large_parcour.csv')

fig, ax = plt.subplots(figsize=(15, 15))

plot_parcour(parcour, ax)
parcour_graph = links_to_graph(parcour)

path_dijkstra, length_dijstra = dijkstra(parcour_graph, (0, 0), (99, 99))
add_path(path_dijkstra, ax, 'red')

path_astar, length_astar = a_star #...
add_path(path_astar, ax, 'blue')

In [ ]:
%timeit path_dijkstra, length_dijstra = dijkstra(parcour_graph, (0, 0), (99, 99))

In [ ]:
%timeit path_astar, length_astar = a_star(parcour_graph, (0, 0), (99, 99), manhattan_distance)

In [ ]:
# a small maze, white lines are the routes that can be followed
# find the shortest path from (0, 0) to (99, 99)
small_maze = load_links('Data/small_maze.csv')

fig, ax = plt.subplots(figsize=(10, 10))

plot_parcour(small_maze, ax, line_width=15)
maze_graph = links_to_graph(small_maze)

path_dijkstra, length_dijstra = dijkstra(maze_graph, (0, 0), (19, 19))
add_path(path_dijkstra, ax, 'red')

path_astar, length_astar = a_star #...
add_path(path_astar, ax, 'blue')

In [ ]:
%timeit path_dijkstra, length_dijstra = dijkstra(maze_graph, (0, 0), (19, 19))

In [ ]:
%timeit path_astar, length_astar = a_star(maze_graph, (0, 0), (19, 19), manhattan_distance)

In [ ]:
# a small maze, white lines are the routes that can be followed
# find the shortest path from (0, 0) to (99, 99)
large_maze = load_links('Data/large_maze.csv')

fig, ax = plt.subplots(figsize=(15, 15))

plot_parcour(large_maze, ax)
maze_graph = links_to_graph(large_maze)

path_dijkstra, length_dijstra = dijkstra(maze_graph, (0, 0), (99, 99))
add_path(path_dijkstra, ax, 'red')

path_astar, length_astar = a_star#...
add_path(path_astar, ax, 'blue')

In [ ]:
%timeit path_dijkstra, length_dijstra = dijkstra(maze_graph, (0, 0), (99, 99))

In [ ]:
%timeit path_astar, length_astar = a_star(maze_graph, (0, 0), (99, 99), manhattan_distance)

## Project: the green city of Ghent

![Citadelpark.](Figures/citadel.jpg)

Ghent is a very pleasant city to live in, a bit of green is never far away. In this small project we will compute the shortest distance to the closest park for every street crossing in Ghent, a problem with tens of thousands of vertices.

Let us take a look at the data.

In [ ]:
import json
ghent_data = json.load(fp=open('Data/graph_parks_ghent.json', 'r'))

In [ ]:
ghent_data.keys()

Below is the graph of streets of Ghent. The vertices correspons to street crossings, the edges to street (segments). The edges are weighted by the length of the street, in km.

In [ ]:
graph_ghent = {k : set(map(tuple, v)) for k, v in ghent_data['adjacency_list'].items()}

In [ ]:
min([min(v) for v in graph_ghent.values()])

Every vertex has an xy-coordinate.

In [ ]:
coordinates = {k : tuple(v) for k, v in ghent_data['coordinates'].items()}

Finally, you receive a set of the the vertices that are in a park!

In [ ]:
park_vertices = set(ghent_data['park vertices'])

You can plot the data below (at own risk, this is a HUGE graph!):

**Assignment 3**

1. Complete the code for finding the distances to the park. This is a modified version of Dijkstra. You are expected to 'design' this algorithm yourself. It returns a dictonary with the distance to the nearest park and a dictionary containing the tree of these paths.
2. Compute and plot these distances.

In [ ]:
def compute_distances_to_park(city_graph, park_vertices):
    """
    Modification of Dijkstra to find the tree with all shortest paths
    to the nearest park
    
    Inputs:
        - city_graph : dict representing the weighted graph of the city
        - park_vertices : the set of vertices laying in a park

    Ouput:
        - distances_to_park : dict with the distances of the nodes to nearest park
        - paths : dict containing for every vertex (key) the next vertex (value) in the 
                path to the closest park
    """
    # COMPLETE THIS
    return distances_to_park, paths
        
    

In [ ]:
dist_to_park, paths = compute_distances_to_park(graph_ghent, park_vertices)
dist_to_park = {k : v for k, v in dist_to_park.items() if v < inf}

In [ ]:
# plot the results